# [IAPR][iapr]: Project ‒  Coin classification


**Group ID:** 56

**Author 1 (sciper):** Shujiro Shobayashi (314853)  
**Author 2 (sciper):** Camille Pittet (326866)   
**Author 3 (sciper):** Renuka Singh Virk (326470) 

**Release date:** 26.04.2024  
**Due date:** 29.05.2024 (11:59 pm)


## Important notes

Each group must upload an archive by May 29th, 11:59 PM that includes:

1. A report of your work in Jupyter Notebook format. It should provide detailed explanations of your method along with the code for every step of the project. The Jupyter Notebook should be capable of generating the final submission you submitted on Kaggle when rerun.
2. A requirements.txt file listing all Python packages you used.
3. Any additional files that you used.
TAs should be able to rerun your code end to end without having any issues. If not, you might lose part of the points during grading.

Please name the archive file with the number of your team, e.g. Group28.zip.

In [ ]:
!pip install numpy
!pip install matplotlib
!pip install scikit-image
!pip install pillow

In [ ]:
# Check is at least python 3.9
import sys 
assert (sys.version_info.major == 3) and (sys.version_info.minor == 9)

In [ ]:
# Import main packages
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from PIL import Image
from typing import Callable
import os
from datetime import datetime

---
# Localisation

---
# Feature extraction

---
# Classification